# NYC Inspections - part I

Initial Data Analysis

The Analysis performed in this notebook resulted in the class `Preprocess` available in `src/preprocess.py` that cleans up and aggregates the relevant information of the nyc inspections dataset

In [1]:
import pandas as pd

pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv('../data/43nn-pn8j.csv')
df.shape

(100000, 27)

In [3]:
df.duplicated().sum()

np.int64(2)

In [4]:
df.drop_duplicates(inplace=True)

In [5]:
# convert date columns to datetime
date_columns = ['inspection_date', 'record_date', 'grade_date'] 
for col in date_columns:
    df[col] = pd.to_datetime(df[col], errors='coerce')

"Records are also included for each restaurant that has applied for a permit but has not yet been inspected and for inspections resulting in no violations. Establishments with inspection date of 1/1/1900 are new establishments that have not yet received an inspection." (from: https://data.cityofnewyork.us/Health/DOHMH-New-York-City-Restaurant-Inspection-Results/43nn-pn8j/about_data)

These entries can be safely removed.

In [6]:
print(f"Initial sample size: {df.shape}")
df = df[df['inspection_date'] != '1900-01-01 00:00:00']
print(f"Sample size after removing 1/1/1900 entries: {df.shape}")

Initial sample size: (99998, 27)
Sample size after removing 1/1/1900 entries: (98293, 27)


In [7]:
print(f"Inspection dates between: {df['inspection_date'].min()} and {df['inspection_date'].max()}")

Inspection dates between: 2015-09-24 00:00:00 and 2025-07-16 00:00:00


"Letter grading inspections were put on pause beginning March 17, 2020, until July 19, 2021, due to the COVID-19 public health emergency. Modified restaurant inspections occurred during this time." [source:Word file from the `/documentation` folder]

In [8]:
# during covid most inspections were suspended
print(f"{df[(df['inspection_date']>= '2020-03-17') & (df['inspection_date'] < '2021-07-19')].sort_values(by='inspection_date').shape[0]} inspections took place during COVID-19 (March 17, 2020 - July 19, 2021)")

# remove these inspections from the dataset
df = df[(df['inspection_date']< '2020-03-17') | (df['inspection_date'] >= '2021-07-19')]
print(f"Sample size after removing COVID-19 inspections: {df.shape}")

9 inspections took place during COVID-19 (March 17, 2020 - July 19, 2021)
Sample size after removing COVID-19 inspections: (98284, 27)


Assumption -> **risk category inferred from Grade**

"Grade associated with the inspection;
 * N = Not Yet Graded
 * A = Grade A
 * B = Grade B
 * C = Grade C
 * Z = Grade Pending
 * P= Grade Pending issued on re-opening following an initial inspection that resulted in a closure
"
from: https://data.cityofnewyork.us/Health/DOHMH-New-York-City-Restaurant-Inspection-Results/43nn-pn8j/about_data


Risk Category:
* Low --> Grade A
* Medium --> Grade B
* High --> Grade C

In [9]:
# Inspect the columns
for col in df.columns:
    print(f"\n{col}: {df[col].dtype}")
    print(f"Null values: {df[col].isnull().sum()}")
    print(f"Unique values in {col}:")
    print(" - Number of unique values:", df[col].nunique())
    print(" - First 5 unique values:", df[col].unique()[:5])


camis: int64
Null values: 0
Unique values in camis:
 - Number of unique values: 24566
 - First 5 unique values: [50078218 41510846 50137711 50040703 50056175]

dba: object
Null values: 0
Unique values in dba:
 - Number of unique values: 19650
 - First 5 unique values: ['MEAL MART' '218 RESTAURANT' 'HONGDAE' 'BEBE FRITAY' 'SUNDAY IN BROOKLYN']

boro: object
Null values: 0
Unique values in boro:
 - Number of unique values: 5
 - First 5 unique values: ['Queens' 'Manhattan' 'Brooklyn' 'Staten Island' 'Bronx']

building: object
Null values: 184
Unique values in building:
 - Number of unique values: 7086
 - First 5 unique values: ['7210' '218220' '14909' '826' '348']

street: object
Null values: 0
Unique values in street:
 - Number of unique values: 2215
 - First 5 unique values: ['MAIN ST' 'GRAND STREET' '41ST AVE' 'ROGERS AVENUE' 'WYTHE AVENUE']

zipcode: float64
Null values: 960
Unique values in zipcode:
 - Number of unique values: 218
 - First 5 unique values: [11367.    nan 11355. 1122

In [10]:
df.groupby('inspection_type')['grade'].value_counts().unstack().join(
    df.groupby('inspection_type').size().rename('total')).reset_index()

,inspection_type,A,B,C,N,P,Z,total
0,Administrative Miscellaneous / Initial Inspection,NaN,NaN,NaN,4.0,NaN,NaN,2963
1,Administrative Miscellaneous / Re-inspection,NaN,NaN,1.0,NaN,NaN,NaN,622
2,Cycle Inspection / Initial Inspection,18923.0,1.0,2.0,562.0,NaN,NaN,53104
3,Cycle Inspection / Re-inspection,9047.0,4995.0,3203.0,10.0,NaN,1426.0,19452
4,Cycle Inspection / Reopening Inspection,3.0,1.0,104.0,1.0,258.0,127.0,967
5,Inter-Agency Task Force / Initial Inspection,NaN,NaN,NaN,8.0,NaN,NaN,292
6,Pre-permit (Non-operational) / Compliance Insp...,NaN,NaN,NaN,5.0,NaN,NaN,15
7,Pre-permit (Non-operational) / Initial Inspection,NaN,NaN,NaN,381.0,NaN,NaN,1237
8,Pre-permit (Non-operational) / Re-inspection,NaN,NaN,NaN,56.0,NaN,NaN,119
9,Pre-permit (Non-operational) / Second Complian...,NaN,NaN,NaN,3.0,NaN,NaN,5


**Gradable Inspections**

Not all types of inspections award a grade (this is in accordance with the documentation)

Inspections that do not award a grade (A, B or C), are not relevant for the task and so, can be removed.

"Identifying Gradable Inspections

Not every inspection is a “gradable” inspection. Gradable inspections can be identified when the following statements are true: 
INSPECTION TYPE in (Cycle Inspection/Initial Inspection, Cycle Inspection/Re-Inspection, Pre-Permit (Operational)/Initial Inspection, Pre-Permit (Operational)/Re-Inspection)
ACTION in (Violations were cited in the following area(s), No violations were recorded at the time of this inspection, Establishment Closed by DOHMH)
INSPECTION DATE >= July 27, 2010" [source:Word file from the `/documentation` folder]

Note: 'Establishment Closed by DOHMH' is actually  'Establishment Closed by DOHMH. Violations were cited in the following area(s) and those requiring immediate action were addressed.' in the dataset

In [11]:
# gradable inspection  
df_gradable_inspection = df[(df['inspection_type'].isin(['Cycle Inspection / Initial Inspection', 
                                                          'Cycle Inspection / Re-inspection', 
                                                          'Pre-permit (Operational) / Initial Inspection', 
                                                          'Pre-permit (Operational) / Re-inspection',
                                                          ]))
                                & (df['action'].isin(['Violations were cited in the following area(s).', 
                                                          'No violations were recorded at the time of this inspection.', 
                                                          ])
                                                          | df['action'].str.contains('Establishment Closed by DOHMH', case=False)
                                )
                                & (df['inspection_date'] >= '2010-07-27')].copy()
df_gradable_inspection.shape

(89910, 27)

In [12]:
df_gradable_inspection['action'].value_counts()

action
Violations were cited in the following area(s).                                                                                       86773
Establishment Closed by DOHMH. Violations were cited in the following area(s) and those requiring immediate action were addressed.     2959
No violations were recorded at the time of this inspection.                                                                             178
Name: count, dtype: int64

In [13]:
df_gradable_inspection['grade'].value_counts()

grade
A    33198
B     5941
C     3863
N     2396
Z     1875
Name: count, dtype: int64

In [14]:
df_gradable_inspection['grade'].isna().sum()

np.int64(42637)

In [15]:
df_gradable_inspection['score'].isna().sum()

np.int64(0)

In the `df_gradable_inspection` dataframe, grades can be (A, B, C, N, Z or NaN).
The `score`column is always filled in.

- For entries with grades N or Z, a grade between A, B or C will be computed from the `score`.
- For entries with NaN, a grade between A, B or C will be computed from the `score`
- For entries with A, B or C, the grade will be checked agains the `score`:
    * if they are aligned, the grade is kept
    * if they are not aligned, the new correct grade is computed from the `score`

In [16]:
df_gradable_inspection.loc[:,'computed_grade'] = df_gradable_inspection['score'].apply(
    lambda x: 'A' if x <= 13 else ('B' if x <= 27 else 'C'))

In [17]:
mask = df_gradable_inspection['grade'].isin(['A', 'B', 'C']) & (df_gradable_inspection['computed_grade'] != df_gradable_inspection['grade'])

print(f"There are {df_gradable_inspection.loc[mask].shape[0]} entries where the grade is not aligned with the score.")
#df_gradable_inspection.loc[mask]

There are 63 entries where the grade is not aligned with the score.


In [18]:
df['grade'].unique()

array([nan, 'B', 'A', 'C', 'N', 'Z', 'P'], dtype=object)

In [19]:
df_gradable_inspection['computed_grade'].value_counts()

computed_grade
A    35975
C    29879
B    24056
Name: count, dtype: int64

In [20]:
df_gradable_inspection.shape

(89910, 28)

In [21]:

assert df_gradable_inspection.groupby(['camis', 'inspection_date'])[['inspection_type', 'action', 'grade', 'violation_code']].nunique().reset_index()['inspection_type'].max() == 1, "There are multiple inspection types  for a single camis/inspection_date in df_gradable_inspection"

assert df_gradable_inspection.groupby(['camis', 'inspection_date'])[['inspection_type', 'action', 'grade', 'violation_code']].nunique().reset_index()['action'].max() == 1, "There are multiple actions for a single camis/inspection_date in df_gradable_inspection"

assert df_gradable_inspection.groupby(['camis', 'inspection_date'])[['inspection_type', 'action', 'grade', 'violation_code']].nunique().reset_index()['grade'].max() == 1, "There are multiple grades for a single camis/inspection_date in df_gradable_inspection"


In [22]:
df_gradable_inspection['violation_code'].isna().sum()

np.int64(177)

In [23]:
df_gradable_inspection['boro'].isna().sum()

np.int64(0)

In [24]:
df_gradable_inspection['council_district'].isna().sum()

np.int64(1150)

In [25]:
df_gradable_inspection['violation_code'].unique()

array(['09B', '04K', '02B', '10J', '04M', '04L', '09E', '06F', '10H',
       '08A', nan, '06I', '10C', '09C', '05A', '10F', '10B', '02G', '05D',
       '10G', '06C', '04J', '06D', '02H', '10A', '10E', '06E', '04A',
       '09A', '04N', '06A', '08C', '05F', '03A', '04O', '04H', '28-06',
       '04D', '04E', '05H', '10D', '03B', '08B', '05E', '28-05', '04F',
       '04C', '02C', '02A', '05C', '02I', '06B', '03I', '10I', '04P',
       '05B', '06G', '03C', '02F', '03F', '06H', '02D', '09D', '03E',
       '28-07', '04B', '07A', '03D', '03G', '18-11', '04I'], dtype=object)

In [26]:
df_gradable_inspection.groupby(['violation_code', 'violation_description']).size()

violation_code  violation_description                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
02A             Food not cooked

In [27]:
# Group and aggregate
df_agg = df_gradable_inspection.groupby(['camis', 'inspection_date']).agg({
    'inspection_type': lambda x: x.unique()[0],
    'computed_grade': lambda x: x.unique()[0],
    'action': lambda x: x.unique()[0],
    'boro': 'first',
    'latitude': 'first',
    'longitude': 'first',
    'critical_flag': lambda x:  list(x),
    'violation_code': lambda x: list(x.dropna())        #an empty list here implies no violations
}).reset_index().sort_values(by=['camis', 'inspection_date'])

df_agg.shape

(53189, 10)

In [28]:
df_agg.head()

,camis,inspection_date,inspection_type,computed_grade,action,boro,latitude,longitude,critical_flag,violation_code
0,30075445,2023-01-31,Cycle Inspection / Initial Inspection,B,Establishment Closed by DOHMH. Violations were...,Bronx,40.848231,-73.855972,[Critical],[06D]
1,30075445,2023-08-22,Cycle Inspection / Re-inspection,A,Violations were cited in the following area(s).,Bronx,40.848231,-73.855972,[Not Critical],[08A]
2,30075445,2024-11-08,Cycle Inspection / Initial Inspection,A,Violations were cited in the following area(s).,Bronx,40.848231,-73.855972,[Not Critical],[10F]
3,30191841,2023-04-23,Cycle Inspection / Initial Inspection,A,Violations were cited in the following area(s).,Manhattan,40.767326,-73.984310,[Critical],[06E]
4,30191841,2024-11-20,Cycle Inspection / Initial Inspection,B,Violations were cited in the following area(s).,Manhattan,40.767326,-73.984310,"[Not Critical, Not Critical]","[10F, 08A]"


In [29]:
df_gradable_inspection['latitude'].isna().sum(), df_gradable_inspection['longitude'].isna().sum()

(np.int64(124), np.int64(124))

In [30]:
df_agg.shape

(53189, 10)